In [1]:
import pandas as pd
from myUtils import pickle_load, pickle_dump
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.width', 790)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

fp_df_picks  = f'df_picks'

In [2]:
df = pickle_load(path_data_dump, fp_df_picks)
df = df.drop_duplicates(subset=['date_end_df_train', 'max_days_lookbacks', 'days_lookbacks'], keep='last')
df = df.sort_values(by=['date_end_df_train', 'max_days_lookbacks', 'days_lookbacks'], ascending=False)
df = df.reset_index(drop=True)
print(f'len(df): {len(df)}')
df

len(df): 28


,date_end_df_train,max_days_lookbacks,days_lookbacks,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,2023-04-03,120,"[30, 60, 120]",[],[],[],[],[],[],[],"['GE', 'NVDA']",[],"['ACLS', 'ANET', 'FTSM', '...","['META', 'OLED', 'SGEN']","['CRUS', 'FCN', 'GBTC', 'P...","['AMPH', 'AVGO', 'BTC-USD']",[]
1,2023-04-03,120,"[15, 30, 60, 120]",[],[],[],[],[],['NVDA'],[],"['GE', 'PERI']",[],"['ACLS', 'ANET', 'FTSM', '...","['AU', 'ELF', 'META', 'OLE...","['CRUS', 'FCN', 'GBTC', 'O...",[],[]
2,2023-03-31,120,"[30, 60, 120]",[],[],[],[],[],[],[],['NVDA'],"['FTSM', 'HY', 'SHV']","['ACLS', 'ANET', 'GE', 'LN...","['META', 'SGEN']","['AMD', 'CRUS', 'FCN', 'GB...","['CNK', 'COTY']",[]
3,2023-03-31,120,"[15, 30, 60, 120]",[],[],[],[],[],['NVDA'],[],[],"['FTSM', 'HY', 'SHV']","['ACLS', 'ANET', 'GBTC', '...","['AMD', 'AU', 'ELF', 'META...","['CRUS', 'FCN', 'MLR', 'NVO']",[],[]
4,2023-03-30,120,"[30, 60, 120]",[],[],[],[],[],[],['SHV'],['NVDA'],"['FTSM', 'HY']","['GE', 'LNTH', 'PDFS']","['ACLS', 'META', 'NVO', 'S...","['AMD', 'AMPH', 'CRUS', 'F...",['AJRD'],[]
5,2023-03-30,120,"[15, 30, 60, 120]",[],[],[],[],[],['NVDA'],['SHV'],[],"['FTSM', 'HY', 'SGEN']","['GBTC', 'GE', 'LNTH', 'PD...","['ACLS', 'AU', 'BTC-USD', ...","['AMD', 'AMPH', 'CRUS', 'F...",[],[]
6,2023-03-29,120,"[30, 60, 120]",[],[],[],[],[],[],['SHV'],[],"['FTSM', 'HY']","['GE', 'LNTH', 'NVDA', 'PD...","['ACLS', 'GBTC', 'PERI', '...","['FCN', 'META', 'MLR', 'NT...",['AJRD'],[]
7,2023-03-29,120,"[15, 30, 60, 120]",[],[],[],[],[],[],['SHV'],['SGEN'],"['FTSM', 'GBTC', 'HY']","['GE', 'LNTH', 'NVDA', 'PD...","['ACLS', 'BTC-USD', 'OR', ...","['AU', 'FCN', 'MARA']",[],[]
8,2023-03-28,120,"[30, 60, 120]",[],[],[],[],[],[],['SHV'],[],"['FTSM', 'HY']","['GE', 'LNTH', 'PDFS', 'SG...","['GBTC', 'MLR', 'NVDA', 'P...","['ACLS', 'AMPH', 'COTY', '...","['CNK', 'ELF']",[]
9,2023-03-28,120,"[15, 30, 60, 120]",[],[],[],[],[],['SHV'],[],['SGEN'],"['FTSM', 'HY', 'MLR', 'TA']","['GE', 'LNTH', 'PDFS', 'SNY']","['GBTC', 'NVDA', 'OR', 'PE...","['ACLS', 'AMPH', 'BTC-USD'...",[],[]


In [3]:
pickle_dump(df, path_data_dump, fp_df_picks)

In [4]:
from itertools import product
from ast import literal_eval

my_cols = ['sym_freq_12', 'sym_freq_9', 'sym_freq_8']
my_days_lookbacks = [[30, 60, 120], [15, 30, 60, 120]]

my_dates = []
for my_days_lookback in my_days_lookbacks:
  my_row = (df['days_lookbacks'] == str(my_days_lookback))
  l_dates = df.loc[my_row, 'date_end_df_train'].tolist()     
  my_dates.append(l_dates)

my_dates = [val for sublist in my_dates for val in sublist]
my_dates = sorted(list(set(my_dates)), reverse=True)

d_lbk_freq = list(product(my_dates, my_days_lookbacks, my_cols))

picks = []
prev_date = ''
# print(f'Model predicts the best performing symbols are:\n[30, 60, 120]        sym_freq_9\n[15, 30, 60, 120]    sym_freq_12\n[30, 60, 120]        sym_freq_8')
print(f'Model predicts the best performing symbols are:')
print(f'1st    {"[30, 60, 120]":<20}{"sym_freq_9":<14}{"days_eval = 4":<20}')
print(f'2nd    {"[15, 30, 60, 120]":<20}{"sym_freq_12":<14}{"days_eval = 5":<20}')
print(f'3rd    {"[30, 60, 120]":<20}{"sym_freq_8":<14}{"days_eval = 4":<20}')
for i, item in enumerate(d_lbk_freq):
  my_date, my_days_lookback, my_col = item[0], item[1], item[2]
  my_row = (df['date_end_df_train'] == my_date) & (df['days_lookbacks'] == str(my_days_lookback))  
  list_in_df = df.loc[my_row, my_col]
  my_str = list_in_df.tolist()[0]
  my_list = literal_eval(my_str)
  blank = ' '
  if my_date != prev_date:
    if prev_date == '':
      print('\nModel Picks:')  
      print(f'{i:<6}{my_date:<14}{str(my_days_lookback):<21}{my_col:<11}: {my_list}')
    else:
      print(f'\n{i:<6}{my_date:<14}{str(my_days_lookback):<21}{my_col:<11}: {my_list}') 
  else:
    print(f'{i:<6}{blank:<14}{str(my_days_lookback):<21}{my_col:<11}: {my_list}')

  picks.append((i, my_list))
  prev_date = my_date

Model predicts the best performing symbols are:
1st    [30, 60, 120]       sym_freq_9    days_eval = 4       
2nd    [15, 30, 60, 120]   sym_freq_12   days_eval = 5       
3rd    [30, 60, 120]       sym_freq_8    days_eval = 4       

Model Picks:
0     2023-04-03    [30, 60, 120]        sym_freq_12: []
1                   [30, 60, 120]        sym_freq_9 : []
2                   [30, 60, 120]        sym_freq_8 : ['GE', 'NVDA']
3                   [15, 30, 60, 120]    sym_freq_12: []
4                   [15, 30, 60, 120]    sym_freq_9 : []
5                   [15, 30, 60, 120]    sym_freq_8 : ['GE', 'PERI']

6     2023-03-31    [30, 60, 120]        sym_freq_12: []
7                   [30, 60, 120]        sym_freq_9 : []
8                   [30, 60, 120]        sym_freq_8 : ['NVDA']
9                   [15, 30, 60, 120]    sym_freq_12: []
10                  [15, 30, 60, 120]    sym_freq_9 : []
11                  [15, 30, 60, 120]    sym_freq_8 : []

12    2023-03-30    [30, 60, 120]   

In [5]:
z_   = zip(d_lbk_freq, picks)
for i, stuff in enumerate(z_):
  _date, _days_lookback, _sym_freq, _syms = stuff[0][0], stuff[0][1], stuff[0][2], stuff[1][1]
  print(i, _date, _days_lookback, _sym_freq, _syms)


0 2023-04-03 [30, 60, 120] sym_freq_12 []
1 2023-04-03 [30, 60, 120] sym_freq_9 []
2 2023-04-03 [30, 60, 120] sym_freq_8 ['GE', 'NVDA']
3 2023-04-03 [15, 30, 60, 120] sym_freq_12 []
4 2023-04-03 [15, 30, 60, 120] sym_freq_9 []
5 2023-04-03 [15, 30, 60, 120] sym_freq_8 ['GE', 'PERI']
6 2023-03-31 [30, 60, 120] sym_freq_12 []
7 2023-03-31 [30, 60, 120] sym_freq_9 []
8 2023-03-31 [30, 60, 120] sym_freq_8 ['NVDA']
9 2023-03-31 [15, 30, 60, 120] sym_freq_12 []
10 2023-03-31 [15, 30, 60, 120] sym_freq_9 []
11 2023-03-31 [15, 30, 60, 120] sym_freq_8 []
12 2023-03-30 [30, 60, 120] sym_freq_12 []
13 2023-03-30 [30, 60, 120] sym_freq_9 ['SHV']
14 2023-03-30 [30, 60, 120] sym_freq_8 ['NVDA']
15 2023-03-30 [15, 30, 60, 120] sym_freq_12 []
16 2023-03-30 [15, 30, 60, 120] sym_freq_9 ['SHV']
17 2023-03-30 [15, 30, 60, 120] sym_freq_8 []
18 2023-03-29 [30, 60, 120] sym_freq_12 []
19 2023-03-29 [30, 60, 120] sym_freq_9 ['SHV']
20 2023-03-29 [30, 60, 120] sym_freq_8 []
21 2023-03-29 [15, 30, 60, 120] sy

In [6]:
import yfinance as yf
df_SPY = yf.Ticker('SPY').history(period='2y')
days_eval = 4
# df_SPY.index[-(days_eval+1):].strftime('%Y-%m-%d')
# df_SPY.index[-4:-1].strftime('%Y-%m-%d')
df_SPY.index
date_idx = sorted(list(set(df_SPY.index.strftime('%Y-%m-%d'))), reverse=True)  # descending
print(date_idx[0:days_eval+1])
print(date_idx[0], date_idx[days_eval])
print(date_idx[0:6])

['2023-04-03', '2023-03-31', '2023-03-30', '2023-03-29', '2023-03-28']
2023-04-03 2023-03-28
['2023-04-03', '2023-03-31', '2023-03-30', '2023-03-29', '2023-03-28', '2023-03-27']


In [7]:
print(date_idx[1:3])
print(date_idx[3:7])

['2023-03-31', '2023-03-30']
['2023-03-29', '2023-03-28', '2023-03-27', '2023-03-24']
